In [25]:
from kafka import KafkaProducer, KafkaConsumer
from kafka import KafkaClient
import os
import io
import random
import avro.schema
from avro.io import DatumWriter
import json
import uuid
import findspark
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.avro.functions import from_avro, to_avro

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.2.0'

os.environ["SPARK_HOME"] = "/opt/spark/"
#os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.2.0,org.apache.kafka:kafka-clients:3.2.0'

findspark.init('~/.local/bin/pyspark')
#findspark.init('~/.local/lib/python3.8/site-packages/pyspark')
#findspark.init('~/.local/lib/python3.10/site-packages/pyspark')

In [26]:
spark = SparkSession.builder.master("local[1]") \
                    .appName('Operacoes') \
                    .getOrCreate()

print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

topic = 'operacoes'
topic_enriquecido = 'test2'
bootstrapServers = 'localhost:9092'

# `from_avro` requires Avro schema in JSON string format.
jsonFormatSchema = open("operacoes.avsc", "r").read()

# .config('spark.jars.packages', 'spark-sql-kafka-0-10_2.13,spark-avro_2.13')\

Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



In [13]:
import mysql.connector
import pandas as pd

df = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", bootstrapServers)\
  .option("subscribe", topic)\
  .option("startingOffsets", "earliest")\
  .load()

#df1 = df.select(from_avro("value", jsonFormatSchema).alias("operations"))

ds = df\
  .selectExpr("CAST(value AS STRING) as json")\
  .writeStream\
  .format("console") \
  .outputMode("append") \
  .start() \
  .awaitTermination()


NameError: name 'spark' is not defined